In [ ]:
! git clone https://github.com/qmlcode/tutorial.git

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
%cd tutorial
! ls

In [ ]:
#!/usr/bin/env python
from __future__ import print_function
import numpy as np

import qml
from qml.kernels import gaussian_kernel
from qml.math import cho_solve

from tutorial_data import compounds
from tutorial_data import energy_pbe0
from tutorial_data import energy_delta

if __name__ == "__main__":

    # For every compound generate a coulomb matrix
    for mol in compounds:

        mol.generate_coulomb_matrix(size=23, sorting="row-norm")
        # mol.generate_bob(size=23, asize={"O":3, "C":7, "N":3, "H":16, "S":1})
            

    # Make a big 2D array with all the 
    X = np.array([mol.representation for mol in compounds], dtype=np.float32)
    energy_pbe0 = np.array(energy_pbe0,  dtype=np.float32)
    # X = np.array([mol.bob for mol in compounds])

    print(energy_pbe0)

    # Assign 1000 first molecules to the training set
    X_training = X[:1000]
    Y_training = energy_pbe0[:1000]

    # Y_training = energy_delta[:1000]

    # Assign 1000 first molecules to the training set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]
    # Y_test = energy_delta[-1000:]
   
    # Calculate the Gaussian kernel
    sigma = 100 #700.0
    K = gaussian_kernel(X_training, X_training, sigma)
    print(K)

    # Add a small lambda to the diagonal of the kernel matrix
    K[np.diag_indices_from(K)] += 1e-8

    # Use the built-in Cholesky-decomposition to solve
    alpha = cho_solve(K, Y_training) 

    #print(alpha)

    # Assign 1000 last molecules to the test set
    X_test = X[-1000:]
    Y_test = energy_pbe0[-1000:]

    # calculate a kernel matrix between test and training data, using the same sigma
    Ks = gaussian_kernel(X_test, X_training, sigma)

    # Make the predictions
    Y_predicted = np.dot(Ks, alpha)

    # Calculate mean-absolute-error (MAE):
    print(np.mean(np.abs(Y_predicted - Y_test)))


In [ ]:
len(X_training)

**Now make build a neural network with the coulomb matrix as input which is getting featurized by a number of
convolutional layers and subsequently we apply a number of fully connected layers to tackle the regression problem**


Note that for application runs you should change 

kears_estimator = KerasRegressor(build_fn=create_model , epochs=2,  batch_size=25, verbose=2)

to 
kears_estimator = KerasRegressor(build_fn=create_model , epochs=2500,  batch_size=25, verbose=2)

to get a reasonable accucacy



After the cross validation (dont be suprised sklearn gives the negative MAE because they use accuracy not loss) you can get the best model
using search.best_estimator_



**you should be able to get down to 15 kcal/mol just like the kernel above or maybe a little lower because the kernel version was not CV optimized on the trainingset**

In [ ]:
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import datetime
from datetime import datetime

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import scipy
from scipy.stats import reciprocal
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, Convolution1D, Dense, MaxPooling1D,MaxPooling2D,AveragePooling2D,
                                    Flatten, Dropout, Activation, average,
                                    BatchNormalization, Reshape, Conv2D)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, QuantileTransformer

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from keras import backend as K


#X           = np.load("rep.npy", allow_pickle=True)
#energy      = np.load("energy.npy", allow_pickle=True)
"""
X_training  = X[:1000]
Y_training  = energy[:1000]
X_test      = X[-1000:]
Y_test      = energy[-1000:]
"""




def create_model(optimizer, dense_nparams, lr, decay, LAMBDA, filters, kernel_size): #, ):
    #filters = 50
    if K.backend() == 'tensorflow':
        K.clear_session()

    init = 'he_normal'
    
    
    regu = tf.keras.regularizers.l2(LAMBDA)


    model = Sequential()
    model.add(Dense(276, input_dim=276, kernel_initializer=init,kernel_regularizer=regu ,activation='linear'))
    #276 = 12 x 23 
    #BatchNormalization()
    
    Reshape((12, 23, 1), input_shape=(276,))
    Conv2D(filters, kernel_size, activation = 'selu', padding = 'same', input_shape= [12, 23, 1] ),
    MaxPooling2D(2)
    Conv2D(filters*2, kernel_size, activation = 'selu', padding = 'same' ),
    MaxPooling2D(2)

    Flatten()
    #BatchNormalization()
    #model.add(exponential_layer)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()

    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()


    #Dropout(rate=0.8)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.5)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.4)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='selu'))
    BatchNormalization()
    #Dropout(rate=0.1)
    model.add(Dense(dense_nparams, kernel_initializer=init, activation='linear'))
    model.add(Dense(1, activation='linear', name='Output_Energy'))

    opt = Adam(lr=lr ,decay=decay)
    model.compile(loss='mae', optimizer=opt)
    
    #model.summary()
    return model

# for production runs
#kears_estimator = KerasRegressor(build_fn=create_model , epochs=2500, batch_size=25, verbose=2)

# for test runs
kears_estimator = KerasRegressor(build_fn=create_model , epochs=2,  batch_size=25, verbose=2)



param_grid = {
    'dense_nparams': [1500, 2000,  2500],
    'optimizer':['Adam', 'RMSprop'], 'lr': [0.00075,0.001, 0.0015, 0.002 ], 'decay': [0.002,0.001, 0.0007], 'LAMBDA':  [1e-8, 1e-7, 1e-6], 'filters':  [200, 250, 300, 400], 'kernel_size':[3, 5, 10, 20, 50, 60, 100] }


kfold_splits = 5

#RandomizedSearchCV
search  = RandomizedSearchCV(estimator=kears_estimator, param_distributions=param_grid, n_iter=10, cv=kfold_splits, verbose=10)





my_callbacks =  [keras.callbacks.EarlyStopping(patience=20), keras.callbacks.TerminateOnNaN() ]
#https://medium.com/@am.benatmane/keras-hyperparameter-tuning-using-sklearn-pipelines-grid-search-with-cross-validation-ccfc74b0ce9f

grid_result = search.fit(X_training, Y_training, callbacks= my_callbacks,validation_data=(X_test, Y_test) ) 




# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print ("rest")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xscale("log")
plt.yscale("log")
plt.xlim(50, 1000)
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
best = model #rnd_search_cv.best_estimator_
pred = best.predict(X_test)
pred = pred.flatten()
import matplotlib.pyplot as plt

#plt.plot(pred, Y_test, "+")
plt.hist(np.abs(pred - np.array(Y_test)))
plt.show()
np.mean(np.abs(pred - Y_test))  